In [1]:
import os,pickle
import numpy as np
import tensorflow as tf
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from process_data import preprocess_conditional_flow_data_mass,simuate_conditional_flow_data_mass
from Model.ConditionalRealNVP import ConditionalRealNVP
from Utils.ObjDict import ObjDict
from Utils.mkdir_p import mkdir_p

In [2]:
# __________________________________________________________________ ||
# Basic configurables
# __________________________________________________________________ ||

input_csv_path = "data/train_mass.npy"
saved_model_path = "output/train_condrealnvp_mass_210113_v1/saved_model_300.h5"
output_dir = os.path.dirname(saved_model_path)
event_size = 1000
ndim = 3
ncond = 1

In [3]:
# __________________________________________________________________ ||
# Load models
# __________________________________________________________________ ||

nf_model = ConditionalRealNVP(num_coupling_layers=5,dim=ndim,ncond=ncond)
samples = nf_model.distribution.sample(event_size)
condition = 1.0 * np.ones((event_size,1))
_,_ = nf_model.predict([samples,condition,])
nf_model.load_weights(saved_model_path)

In [4]:
arr = np.load(input_csv_path)
arr_list = preprocess_conditional_flow_data_mass(arr)

In [9]:
# __________________________________________________________________ ||
# Make plots for different conditions
# __________________________________________________________________ ||

n_dim = 5
mass_grid = [arr_list[idx_mass] for idx_mass in np.random.randint(0,len(arr_list),n_dim*n_dim)]
mass_grid.sort(key=lambda x: x.condition[0])
figsize = (50,50)

samples = nf_model.distribution.sample(event_size)
fig_pt1,ax_pt1 = plt.subplots(n_dim,n_dim,figsize=figsize)
fig_pt2,ax_pt2 = plt.subplots(n_dim,n_dim,figsize=figsize)
fig_mll,ax_mll = plt.subplots(n_dim,n_dim,figsize=figsize)

for i,m in enumerate(mass_grid):
    
    ix = int(i / n_dim)
    iy = i % n_dim
    
    condition_str = str(m.condition[0])
    condition = np.ones((event_size,1)) * m.condition[0]

    idx_batch = np.random.randint(0,m.x.shape[0],event_size)
    
    x_true = m.x[idx_batch]
    x_gen,_ = nf_model.predict([samples,condition,])

    ax_pt1[ix,iy].hist(x_true[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True '+condition_str)
    ax_pt1[ix,iy].hist(x_gen[:,0],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow '+condition_str)
    ax_pt1[ix,iy].legend(loc='best')
    ax_pt1[ix,iy].set_title(condition_str)
    
    ax_pt2[ix,iy].hist(x_true[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True '+condition_str)
    ax_pt2[ix,iy].hist(x_gen[:,1],bins=100,density=1.,histtype='step',range=[-10.,10.],label='Flow '+condition_str)
    ax_pt2[ix,iy].legend(loc='best')
    ax_pt2[ix,iy].set_title(condition_str)
    
    ax_mll[ix,iy].hist(x_true[:,2],bins=100,density=1.,histtype='step',range=[-10.,10.],label='True '+condition_str)
    ax_mll[ix,iy].hist(x_gen[:,2],bins=100,density=1.,histtype='step',range=[-5.,5.],label='Flow '+condition_str)
    ax_mll[ix,iy].legend(loc='best')
    ax_mll[ix,iy].set_title(condition_str)
    
fig_pt1.savefig(os.path.join(output_dir,'pt1.png'))
fig_pt2.savefig(os.path.join(output_dir,'pt2.png'))
fig_mll.savefig(os.path.join(output_dir,'mll.png'))

In [32]:
# __________________________________________________________________ ||
# Make plots for likelihood
# __________________________________________________________________ ||

import time

n_dim = 5
mass_grid = [arr_list[idx_mass] for idx_mass in np.random.randint(0,len(arr_list),n_dim*n_dim)]
mass_grid.sort(key=lambda x: x.condition[0])

plot_low = -1.5
plot_high = 1.5
n_grid = 10
x_grid = [plot_low+(plot_high-plot_low)/n_grid*i for i in range(n_grid+1)]

event_size = 5000

z = np.zeros(n_grid+1)
fig, ax = plt.subplots(n_dim,n_dim,figsize=figsize)

for i,m in enumerate(mass_grid):
    
    print("-"*100)
    print("Drawing plot ",i," with mass ",m.condition[0])
    
    ix = int(i / n_dim)
    iy = i % n_dim
    
    start_time = time.time()
    
    condition_str = str(m)

    nf_model.direction = -1
    
    idx_batch = np.random.randint(0,m.x.shape[0],event_size)
    
    condition_concat = np.concatenate([np.ones((event_size,1)) * x for ix,x in enumerate(x_grid)])
    x_data_concat = np.concatenate([m.x[idx_batch] for ix,x in enumerate(x_grid)])
    
    z_concat = nf_model.batch_log_loss([x_data_concat,condition_concat])
    
    for ig,x in enumerate(x_grid):
        z[ig] = tf.reduce_mean(z_concat[ig*idx_batch.shape[0]:(ig+1)*idx_batch.shape[0]])

    ax[ix,iy].plot(x_grid,z,)
    ylims = ax[ix,iy].get_ylim()
    ax[ix,iy].arrow(m.condition[0], ylims[1], 0., ylims[0]-ylims[1],)
    
    elapsed_time = time.time() - start_time
    print("Time used: "+str(elapsed_time)+"s")
    
fig.savefig(os.path.join(output_dir,'log_loss.png'))

----------------------------------------------------------------------------------------------------
Drawing plot  0  with mass  [-0.97735305]


/Users/lucien/.pyenv/versions/3.7.6/lib/python3.7/site-packages/matplotlib/patches.py:1338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  verts = np.dot(coords, M) + (x + dx, y + dy)


Time used: 3.053668975830078s
----------------------------------------------------------------------------------------------------
Drawing plot  1  with mass  [-0.96899617]


/Users/lucien/.pyenv/versions/3.7.6/lib/python3.7/site-packages/matplotlib/patches.py:1338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  verts = np.dot(coords, M) + (x + dx, y + dy)


Time used: 2.5634102821350098s
----------------------------------------------------------------------------------------------------
Drawing plot  2  with mass  [-0.91823386]


...

Time used: 2.8417069911956787s
----------------------------------------------------------------------------------------------------
Drawing plot  23  with mass  [0.90375983]


/Users/lucien/.pyenv/versions/3.7.6/lib/python3.7/site-packages/matplotlib/patches.py:1338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  verts = np.dot(coords, M) + (x + dx, y + dy)


Time used: 2.9515180587768555s
----------------------------------------------------------------------------------------------------
Drawing plot  24  with mass  [0.91052179]


/Users/lucien/.pyenv/versions/3.7.6/lib/python3.7/site-packages/matplotlib/patches.py:1338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  verts = np.dot(coords, M) + (x + dx, y + dy)


Time used: 2.9235470294952393s
